# 체크 포인트 저장 및 불러오기 
- 체크포인트를 저장할 때는 state_dict 이상의 것을 저장해야 함 
- 학습 중에 갱신되는 퍼버와 매개변수들을 포함한 Optimizer의 state_dict을 함께 저장하는 것이 중요 
- 중단 시점의 epoch, 마지막으로 기록된 training loss 등 저장

### 개요 
- 여러 체크포인트들을 저장하기 위해서는 dictionary에 체크포인트를 구성하고 torch.save()를 사용하여 사전을 serialize 해야 함 
- pytorch에서는 .tar 확장자를 사용하는 것이 일반적 
- 항목을 불러올 때는 모델과 옵티마이저를 초기화 하고 torch.load()로 불러옴 

### 단계 
1. 데이터를 불러올 떄 필요한 라이브러리 import 
2. 신경망을 구성하고 초기화
3. 옵티마이저 초기화
4. 일반적인 체크포인트 저장 
5. 일반적인 체크포인트 불러오기

In [3]:
import torch 
import torch.nn as nn 
import torch.optim as optim
import torch.nn.functional as F

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forword(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x 

net = Net() 
print(net)

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [5]:
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [6]:
EPOCHS = 5 
PATH = 'model.pt'
LOSS = 0.4 

torch.save({
            'epoch': EPOCHS,
            'model_state_dict': net.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': LOSS,
            }, PATH)

In [7]:
model = Net()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

model.eval()

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [8]:
model.train()

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)